In [1]:
import pandas as pd

# Load the dataset
file_path = 'amazon.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df.head()


,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import contractions

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Contraction map
contraction_map = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
    "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
    "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
    "i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have",
    "i'm": "I am", "i've": "I have", "isn't": "is not", "it'd": "it would",
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
    "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
    "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock",
    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
    "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
    "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "that'd": "that would",
    "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have",
    "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
    "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
    "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
    "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
    "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
    "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
    "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
    "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have"
}

# Function to expand contractions
def expand_contractions(text, contraction_map):
    pattern = re.compile(r'\b(' + '|'.join(contraction_map.keys()) + r')\b')
    expanded_text = pattern.sub(lambda x: contraction_map[x.group()], text)
    return expanded_text

df['text2'] = contractions.fix(df['reviewText'])

# Function to preprocess text
def preprocess_text(text):
    # 1. Expand Contractions
    text = expand_contractions(text, contraction_map)
    
    # 2. Remove URLs and Emails
    text = re.sub(r'http\S+|www\S+|https\S+|mailto:\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    
    # 3. Remove special characters and emojis
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 4. Tokenization
    words = word_tokenize(text)
    
    # 5. Lowercasing
    words = [word.lower() for word in words]
    
    # 6. Removing Punctuation
    words = [word for word in words if word.isalnum()]
    
    # 7. Removing Stop Words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # 8. Stemming
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    
    # 9. Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return ' '.join(lemmatized_words)

# Apply preprocessing to the dataset
df['preprocessed_text'] = df['text2'].apply(preprocess_text)

# Display the first few rows of the preprocessed dataset
df[['reviewText', 'preprocessed_text']].head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


AttributeError: 'Series' object has no attribute 'lower'

In [5]:
import pandas as pd
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
file_path = 'amazon.csv'
df = pd.read_csv(file_path)

# Function to preprocess text
def preprocess_text(text):
    # 1. Expand Contractions
    text = contractions.fix(text)
    
    # 2. Remove URLs and Emails
    text = re.sub(r'http\S+|www\S+|https\S+|mailto:\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    
    # 3. Remove special characters and emojis
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 4. Tokenization
    words = word_tokenize(text)
    
    # 5. Lowercasing
    words = [word.lower() for word in words]
    
    # 6. Removing Punctuation
    words = [word for word in words if word.isalnum()]
    
    # 7. Removing Stop Words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # 8. Stemming
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    
    # 9. Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return ' '.join(lemmatized_words)

# Apply preprocessing to the dataset
df['preprocessed_text'] = df['reviewText'].apply(preprocess_text)

# Display the first few rows of the preprocessed dataset
print(df[['reviewText', 'preprocessed_text']].head())

# Perform vectorization
# Using CountVectorizer
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(df['preprocessed_text'])
print("Count Vectorizer - Feature Names:", count_vectorizer.get_feature_names_out())
print("Count Vectorizer - Vectorized Text:", count_vector.toarray())

# Using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(df['preprocessed_text'])
print("TF-IDF Vectorizer - Feature Names:", tfidf_vectorizer.get_feature_names_out())
print("TF-IDF Vectorizer - Vectorized Text:", tfidf_vector.toarray())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\satch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                          reviewText  \
0  This is a one of the best apps acording to a b...   
1  This is a pretty good version of the game for ...   
2  this is a really cool game. there are a bunch ...   
3  This is a silly game and can be frustrating, b...   
4  This is a terrific game on any pad. Hrs of fun...   

                                   preprocessed_text  
0  one best apps acording bunch people agree bomb...  
1  pretty good version game free lot different le...  
2  really cool game bunch level find golden egg s...  
3  silly game frustrating lot fun definitely reco...  
4  terrific game pad hr fun grandkids love great ...  
Count Vectorizer - Feature Names: ['aa' 'aaa' 'aaaa' ... 'zzz' 'zzzz' 'zzzzz']
Count Vectorizer - Vectorized Text: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
TF-IDF Vectorizer - Feature Names: ['aa' 'aaa' 'aaaa' ... 'zzz' 'zzzz' 'zzzzz']
TF-IDF Vectorizer 

In [6]:
df['preprocessed_text']

0        one best apps acording bunch people agree bomb...
1        pretty good version game free lot different le...
2        really cool game bunch level find golden egg s...
3        silly game frustrating lot fun definitely reco...
4        terrific game pad hr fun grandkids love great ...
                               ...                        
19995    app fricken stupidit froze kindle allow place ...
19996    please add need neighbor ginger thanks bunch a...
19997    love game awesome wish free stuff house cost m...
19998    love love love app side fashion story fight wo...
19999    game rip list thing make betterbull first need...
Name: preprocessed_text, Length: 20000, dtype: object

In [7]:
processed_file_path = 'amazon_processed.csv'
df.to_csv(processed_file_path, index=False)